# Setting

In [1]:
import requests
import pandas as pd
import time
from tqdm import tqdm

In [2]:
dir_path = '/content/drive/MyDrive/Git/League_of_Legends_Match_Prediction'
with open(dir_path+"/Data/api_key") as f: api_key = f.readline()

In [3]:
def get_league_summonerInfo(league = 'challenger'):
    """
    Get the Summoner ID Series from Riot api server.

    Parameters
    ----------
    league : string

    Returns
    ----------
    Series
    """
    league_list = ['challenger','grandmaster','master']
    if league not in league_list:
        raise ValueError("'{}' is not a valid value for name; supported values are {}".format(league, ', '.join(league_list)))

    url_ = 'https://kr.api.riotgames.com/lol/league/v4/'+league+'leagues/by-queue/RANKED_SOLO_5x5/' + '?api_key=' + api_key
    json_ = requests.get(url_).json()
    summonerId = pd.DataFrame(json_['entries']).loc[:,['summonerId','summonerName']]

    return summonerId

In [4]:
def get_puuid(summonerName):
    """
    Get the puuid Series from Riot api server.

    Parameters
    ----------
    summonerName : string

    Returns
    ----------
    string
    """
    url_ = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + summonerName + '?api_key=' + api_key
    request_ = requests.get(url_)

    while request_.status_code == 429:
        time.sleep(5)
        request_ = requests.get(url_)
    
    if request_.status_code == 200:
        json_ = request_.json()
        ret_ = json_['puuid']
    else:
        ret_ = None

    return ret_

In [5]:
def get_timeline(matchId):
    """
    Get a match timeline from Riot api server.

    Parameters
    ----------
    matchId : string

    Returns
    ----------
    Dict
    """
    url_ = 'https://asia.api.riotgames.com/lol/match/v5/matches/' +matchId + '/timeline' + '?api_key=' + api_key
    request_ = requests.get(url_)
    json_ = request_.json()

    return json_

In [6]:
def get_matchId(puuid):
    """
    Get a matchId from Riot api server.

    Parameters
    ----------
    matchId : string

    Returns
    ----------
    Dict
    """
    count = 20

    url_ = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid +'/ids?start=0&count=' + count + '?api_key=' + api_key
    print(url_)
    request_ = requests.get(url_)
    
    json_ = request_.json()
    
    return json_

In [7]:
challenger_info = get_league_summonerInfo('challenger')
grandmaster_info = get_league_summonerInfo('grandmaster')
top1000_info = pd.concat([challenger_info,grandmaster_info],ignore_index=True)

In [8]:
tqdm.pandas()
top1000_info['puuId'] = top1000_info['summonerName'].progress_apply(get_puuid)

100%|██████████| 1000/1000 [24:48<00:00,  1.49s/it]


In [9]:
top1000_info.to_csv(dir_path+'/Data/puuid.csv',index=False,encoding='utf-8')

In [10]:
top1000_info = pd.read_csv(dir_path+'/Data/puuid.csv')